In [4]:
import math
import random
import gymnasium as gym
import numpy as np
from stable_baselines3.common.env_checker import check_env
print("No problems")

No problems


In [5]:
class Learner_MW:
    def __init__(self, num_actions):
        self.num_actions = num_actions
        self.weights = [1] * num_actions  # Initial weights for all actions

    def choose_action(self):
        total_weight = sum(self.weights)
        probabilities = [weight / total_weight for weight in self.weights]
        action = random.choices(range(self.num_actions), probabilities)[0]
        return action

    #MW update with a fixed learning rate, this is the theoretical object we study, although having len(transcript) instead provides better results 
    def update_weights(self, action, reward, transcript, total_time):
        eta = math.sqrt(2 * math.log(self.num_actions) / total_time)
        for i in range(self.num_actions):
            self.weights[i] *= math.exp(eta * reward[i])  # Update weights based on counterfactual payoff

    def action_probs_cumulative(self, cumulative, total_time):
        if len(cumulative) != self.num_actions:
            return "Serious error"
        eta = math.sqrt(2 * math.log(self.num_actions) / total_time)
        weights = [1] * self.num_actions  # Initial weights for all actions
        for i in range(self.num_actions):
            weights[i] = math.exp(eta * cumulative[i] )  # Update weights based on counterfactual payoff
        total_weight = sum(weights)
        probabilities = [weight / total_weight for weight in weights]
        #The function has been tested
        #print("Probabilities = ", probabilities)
        return probabilities

In [6]:


class SimplexSpace(gym.spaces.Space):
    """
    Defines the action space as a probability simplex over m outcomes.
    """

    def __init__(self, m):
        """
        Initializes the SimplexSpace.

        Parameters:
            - m (int): Number of outcomes.
        """
        assert m >= 2, "Number of outcomes 'm' must be at least 2."
        super(SimplexSpace, self).__init__(shape=(m,), dtype=np.float32)
        self.m = m

    def sample(self):
        """
        Generates a random sample from the simplex space.
        """
        sample = np.random.dirichlet(np.ones(self.m))
        return sample


    def contains(self, x):
        """
        Checks if a given point is within the simplex space.

        Parameters:
            - x (array-like): Point to be checked.

        Returns:
            - bool: True if the point is within the space, False otherwise.
        """
        if len(x) != self.m:
            return False
        if np.sum(x) != 1.0:
            return False
        if np.any(x < 0):
            return False
        return True

    def seed(self, seed=None):
        """
        Seeds the pseudo-random number generator for this space.

        Parameters:
            - seed (int or None): The seed to use. If None, a random seed will be chosen.

        Returns:
            - list[int]: The list of seeds used for seeding the PRNG.
        """
        if seed is None:
            seed = np.random.randint(0, 2**32 - 1)
        np.random.seed(seed)
        return [seed]


In [7]:
bla = SimplexSpace(3)

sample = bla.sample()
print(sample)
print(sample[1])

[0.62260179 0.0644223  0.3129759 ]
0.06442230308329984


In [8]:
class MWBR(gym.Env):
    def __init__(self, learner, payoff_matrix_learner, payoff_matrix_optimizer, num_rounds):
        super(MWBR, self).__init__()

        self.timestep = 0
        
        self.learner  = learner
        self.num_learner_actions = len(payoff_matrix_learner)
        self.num_optimizer_actions = len(payoff_matrix_learner[0])
        self.payoff_matrix_learner = payoff_matrix_learner
        self.payoff_matrix_optimizer = payoff_matrix_optimizer
        self.num_rounds = num_rounds #Total number of rounds
        high = max([max(payoff_matrix_learner[action_learner]) for action_learner in range(learner.num_actions)])
        low = min(0, min([min(payoff_matrix_learner[action_learner]) for action_learner in range(learner.num_actions)]))



        # Define your MDP parameters here
        
        
        #The action space is now the  m-dimensional probability simplex
        #self.action_space = SimplexSpace(self.num_optimizer_actions)
        self.action_space = gym.spaces.Discrete(self.num_optimizer_actions)

        point_space =  gym.spaces.Box(low=np.array([low] * self.num_learner_actions), high=np.array([high] * self.num_learner_actions), dtype=np.float32)
        timestep_space = gym.spaces.Discrete(self.num_rounds+2)

        # Combine subspaces into a tuple space
        #self.observation_space = gym.spaces.Tuple((point_space, timestep_space))


        #observation_space = Dict({"position": Box(-1, 1, shape=(2,)), "color": Discrete(3)}, seed=42)

        self.observation_space = gym.spaces.Dict({"cumulative":point_space, "timestamp" : timestep_space})
        #self.observation_space = (point_space, timestep_space)
        
        
        
        
        #self.observation = gym.spaces.Tuple((np.zeros(self.num_learner_actions, dtype=np.float32),0))
        #self.observation = (np.zeros(self.num_learner_actions, dtype=np.float32), 0)

        self.observation = { 'cumulative': np.zeros(self.num_learner_actions, dtype=np.float32),  # Example initial values for subspace1
                                'timestamp': 0  # Example initial values for subspace2    
                             }

        
        #self.observation_space = spaces.Discrete(num_states)
        self.terminated = False
        self.truncated = False
        # Define transition dynamics, rewards, etc.

    def step(self, action):


        

        # Implement the step function
        # Returns next_state, reward, done, info


        #Unroll the observation space/ state
        cumulative, self.timestep = self.observation

        cumulative = self.observation['cumulative']
        self.timestep = self.observation['timestamp']

        self.timestep +=1
        if self.timestep == self.num_rounds:
            self.terminated = True

        learner_probs = learner.action_probs_cumulative(cumulative,self.num_rounds)

        
                
        #reward = sum(action[optimizer_index] * sum(payoff_matrix_optimizer[action_learner][optimizer_index] * learner_probs[action_learner] for action_learner in range(self.num_learner_actions)) for optimizer_index in range(self.num_optimizer_actions))
        
        reward = sum(payoff_matrix_optimizer[action_learner][action] * learner_probs[action_learner]
                                   for action_learner in range(self.num_learner_actions))


        #new_cumulative = [cumulative[action_learner]+ sum(action[optimizer_index] * payoff_matrix_learner[action_learner][optimizer_index ] for optimizer_index in range(self.num_optimizer_actions) ) for action_learner in range(self.num_learner_actions)]
        
        new_cumulative = [cumulative[action_learner]+ payoff_matrix_learner[action_learner][action] for action_learner in range(self.num_learner_actions)]

        new_cumulative = np.array(new_cumulative, dtype=np.float32)

        #self.observation =(new_cumulative,self.timestep)
        
        self.observation = { 'cumulative': new_cumulative,  # Example initial values for subspace1
                                'timestamp': self.timestep  # Example initial values for subspace2    
                             }


        info = {}
        return self.observation, reward, self.terminated, self.truncated, info

    def reset(self, seed =  None):


        

        # Implement reset function

        self.terminated = False
        self.truncated = False
        self.timestep = 0
        
        #self.observation = gym.spaces.Tuple((np.zeros(self.num_learner_actions, dtype=np.float32),0))
        #self.observation = (np.zeros(self.num_learner_actions, dtype=np.float32), 0)
        self.observation = { 'cumulative': np.zeros(self.num_learner_actions, dtype=np.float32),  # Example initial values for subspace1
                                'timestamp': 0  # Example initial values for subspace2    
                             }

        info = {}
        return self.observation, info

    def render(self, mode='human'):
        # Implement render function
        pass


In [9]:
payoff_matrix_learner = [[0, -1, 1], [1, 0, -1], [-1, 1, 0]]
payoff_matrix_optimizer = [[0, 1, -1], [-1, 0, 1], [1, -1, 0]]

#payoff_matrix_learner = [[0.0005,0], [-1/6, 1/3], [-1/2, 1/2]]
#payoff_matrix_optimizer = [[0,0],[1,0],[0,0]]

#payoff_matrix_learner = [[3,0],[5,1]]
#payoff_matrix_optimizer = [[3,5], [0, 1]]

num_rounds = 1000

learner = Learner_MW(len(payoff_matrix_learner))
game_env = MWBR(learner, payoff_matrix_learner, payoff_matrix_optimizer, 100)

# Use check_env to validate the environment
check_env(game_env)

In [10]:
env = MWBR(learner, payoff_matrix_learner, payoff_matrix_optimizer, num_rounds)
episodes = 50

done = False
count = 0

for episode in range(episodes):
	done = False
	obs = env.reset()
	while not done:#not done:
		random_action = env.action_space.sample()
		print("action",random_action)
		obs, reward, done, truncated, info = env.step(random_action)
		print('reward',reward)


action 1
reward 0.0
action 0
reward 0.015985137930297877
action 1
reward -0.015253131848014745
action 0
reward 0.0
action 2
reward 0.062408040030331735
action 2
reward 0.031238269778312622
action 2
reward 0.0
action 2
reward -0.031238269778312622
action 0
reward 0.02974241557373397
action 0
reward 0.0
action 0
reward -0.02974241557373397
action 0
reward -0.05942585165189107
action 1
reward 0.08899183761855184
action 1
reward 0.05942585165189107
action 0
reward -0.062408040030331735
action 0
reward -0.09344146048097046
action 0
reward -0.12427196596802492
action 2
reward 0.08644841293011363
action 2
reward 0.051961496720692224
action 2
reward 0.017335959705155113
action 2
reward -0.017335959705155113
action 1
reward 0.1363291852606593
action 2
reward -0.03401600492004908
action 1
reward 0.12427196596802492
action 1
reward 0.09344146048097046
action 2
reward -0.032665624456597764
action 1
reward 0.07974251503520197
action 2
reward -0.04791870623760014
action 0
reward 0.014489433587635991

In [11]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.ppo.policies import MultiInputPolicy